In [2]:
from PIL import Image, ImageDraw, ImageFont
import re
import os
import config  # Importing the config file for paths

# Function to parse the corners and bounding box information from the string
def parse_corners_and_bbox(input_string):
    corners_pattern = r"\(([-\d.]+),\s*([-\d.]+)\)"
    corners_matches = re.findall(corners_pattern, input_string)
    corners = [(float(x), float(y)) for x, y in corners_matches[:8]]  # First 8 coordinates are the corners

    bbox_center_pattern = r"Bounding Box Center:\s*\(([-\d.]+),\s*([-\d.]+)\)"
    bbox_center_match = re.search(bbox_center_pattern, input_string)
    bbox_center = (float(bbox_center_match.group(1)), float(bbox_center_match.group(2)))

    bbox_size_pattern = r"Bounding Box Size:\s*Width\s*=\s*([-\d.]+),\s*Height\s*=\s*([-\d.]+)"
    bbox_size_match = re.search(bbox_size_pattern, input_string)
    bbox_width = float(bbox_size_match.group(1))
    bbox_height = float(bbox_size_match.group(2))

    return corners, bbox_center, bbox_width, bbox_height

# Function to draw the skeleton on an image and save it to the validation path
def draw_skeleton_on_images(input_folder, output_folder, input_string):
    # Get all image files from the input folder
    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

    # Ensure output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_file in image_files:
        # Load the image
        image_path = os.path.join(input_folder, image_file)
        img = Image.open(image_path)
        draw = ImageDraw.Draw(img)

        # Parse corners and bounding box
        corners, bbox_center, bbox_width, bbox_height = parse_corners_and_bbox(input_string)

        # Define colors and sizes
        center_radius = 5
        font_size = 20
        green = (0, 255, 0)
        red = (255, 0, 0)
        blue = (0, 0, 255)

        try:
            # Load a font (adjust if necessary)
            font = ImageFont.truetype("arial.ttf", font_size)
        except IOError:
            # Fallback to default PIL font if Arial is unavailable
            font = ImageFont.load_default()

        # Draw the bounding box center (in red)
        bbox_center_x, bbox_center_y = bbox_center
        draw.ellipse((bbox_center_x - center_radius, bbox_center_y - center_radius,
                      bbox_center_x + center_radius, bbox_center_y + center_radius),
                     outline=red, fill=red)

        # Draw the 2D bounding box (in red)
        half_width = bbox_width / 2
        half_height = bbox_height / 2
        top_left = (bbox_center_x - half_width, bbox_center_y - half_height)
        bottom_right = (bbox_center_x + half_width, bbox_center_y + half_height)
        draw.rectangle([top_left, bottom_right], outline=red, width=2)

        # Label each corner with its index (in blue)
        for i, corner in enumerate(corners):
            x, y = corner
            draw.text((x + 5, y - 5), str(i), fill=blue, font=font)  # Blue labels for corners

        # Draw top square (in green)
        draw.line([corners[0], corners[1]], fill=green, width=2)
        draw.line([corners[1], corners[2]], fill=green, width=2)
        draw.line([corners[2], corners[3]], fill=green, width=2)
        draw.line([corners[3], corners[0]], fill=green, width=2)

        # Draw bottom square (in green)
        draw.line([corners[4], corners[5]], fill=green, width=2)
        draw.line([corners[5], corners[6]], fill=green, width=2)
        draw.line([corners[6], corners[7]], fill=green, width=2)
        draw.line([corners[7], corners[4]], fill=green, width=2)

        # Draw vertical edges (in green)
        draw.line([corners[0], corners[4]], fill=green, width=2)  # Top Front Right -> Bottom Front Right
        draw.line([corners[1], corners[5]], fill=green, width=2)  # Top Back Right -> Bottom Back Right
        draw.line([corners[2], corners[6]], fill=green, width=2)  # Top Back Left -> Bottom Back Left
        draw.line([corners[3], corners[7]], fill=green, width=2)  # Top Front Left -> Bottom Front Left

        # Save the modified image to the output folder
        output_image_path = os.path.join(output_folder, f"output_{image_file}")
        img.save(output_image_path)
        print(f"Image saved at {output_image_path}")

# Example usage with paths from config
input_folder = config.get_synthetic_data_test_path()  # Input folder from config
output_folder = config.get_synth_validation_test_path()  # Output folder from config

input_string = """
Projected 2D Corners (x, y):
(407.82, 93.66)
(346.92, 73.81)
(457.62, 66.01)
(541.45, 78.94)
(423.59, 359.05)
(375.58, 287.02)
(456.77, 254.04)
(516.23, 307.12)
Bounding Box Center: (444.19, 212.53)
Bounding Box Size: Width = 244.53, Height = 343.04
"""

draw_skeleton_on_images(input_folder, output_folder, input_string)


Image saved at C:\Users\sakar\OneDrive\mt-datas\synth_validation_data\0_test\output_0.png
